# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
from time import time

from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
#import re
import udacourse2 #my library!
import math

In [2]:
gen_begin = time()

# load messages dataset
messages = pd.read_csv('messages.csv', index_col='id')
#messages['id'] = messages.index
#messages.info()
#messages.index
messages.head()

,message,original,genre
id,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
# load categories dataset
categories = pd.read_csv('categories.csv', index_col='id')
#categories.info()
categories.head()

,categories
id,
2,related-1;request-0;offer-0;aid_related-0;medi...
7,related-1;request-0;offer-0;aid_related-1;medi...
8,related-1;request-0;offer-0;aid_related-0;medi...
9,related-1;request-1;offer-0;aid_related-1;medi...
12,related-1;request-0;offer-0;aid_related-0;medi...


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed
- Index duplicated [here](https://stackoverflow.com/questions/35084071/concat-dataframe-reindexing-only-valid-with-uniquely-valued-index-objects)

In [4]:
#messages.index.duplicated(keep='first')
#drop index duplicated messages
messages = messages.loc[~messages.index.duplicated(keep='first')]

In [5]:
#check number of remaining duplicated messages
print(messages[messages.duplicated()].shape[0])
#drop duplicates
messages = messages.drop_duplicates()
print(messages.shape[0])
# check number of duplicates
messages[messages.duplicated()].shape[0]

3
26177


0

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

- used SQL-type of relations, having the messages dataframe as refference
- documentation [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html)

In [6]:
# merge datasets
df = pd.merge(messages, categories, left_index=True, right_index=True, how='left')
print(df.shape[0])
df.name = 'df'
df.head()

26245


,message,original,genre,categories
id,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26245 entries, 2 to 30265
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   message     26245 non-null  object
 1   original    10184 non-null  object
 2   genre       26245 non-null  object
 3   categories  26245 non-null  object
dtypes: object(4)
memory usage: 1.0+ MB


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

Solution [here](https://stackoverflow.com/questions/42049147/convert-list-to-pandas-dataframe-column)

In [8]:
# create a dataframe of the 36 individual category columns
jumber = df['categories'].iloc[0]
categories = jumber.split(sep=';')
#{'categories': categories}
categories = pd.DataFrame({'categories': categories})
categories.head()

,categories
0,related-1
1,request-0
2,offer-0
3,aid_related-0
4,medical_help-0


In [9]:
# select the first row of the categories dataframe
row = categories.iloc[0]
row['categories'][:-2]

[cat[:-2] for cat in categories['categories']]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = [cat[:-2] for cat in categories['categories']]
print(category_colnames)

# rename the columns of `categories`
#categories.columns = category_colnames
#categories.head()

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


An alert flag, if there was no category inserted:

In [10]:
df['if_blank'] = False

In [11]:
#adding new columns with zero value
for colname in category_colnames:
    df[colname] = 0

#df.columns
print('new shape:',df.shape[1])
#df.head(1)

new shape: 41


In [12]:
cell = df['categories'].iloc[0]
alfa = set(cell.split(sep=';'))
alfa

for beta in alfa:
    if beta.find('1') != -1:
        print(beta[:-2])

related


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [13]:
begin = time()

filtered_cols = ['categories', 'if_blank', 'related', 'request', 'offer', 'aid_related', 'medical_help', 
       'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 
       'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

df[filtered_cols] = df[filtered_cols].apply(lambda x: udacourse2.fn_test(x, verbose=True), axis=1)

spent = time() - begin
print('elapsed time: {:.1f}s ({}min, {:.4f}sec)'.format(spent, math.trunc((spent)/60), math.fmod(spent, 60)))
#df.head()

###
related
###
related
weather_related
aid_related
storm
other_aid
###
related
###
medical_products
related
aid_related
hospitals
buildings
other_aid
request
infrastructure_related
###
related
###
*empty element*
###
related
weather_related
storm
###
related
shelter
water
aid_related
direct_report
request
###
*empty element*
###
medical_products
related
aid_related
direct_report
request
medical_help
###
food
medical_products
related
weather_related
other_infrastructure
water
transport
aid_related
buildings
direct_report
other_aid
floods
request
infrastructure_related
medical_help
###
*empty element*
###
related
water
aid_related
direct_report
request
###
food
related
aid_related
direct_report
request
###
related
###
food
medical_products
related
clothing
shelter
water
aid_related
direct_report
request
###
food
related
water
aid_related
direct_report
request
medical_help
###
*empty element*
###
*empty element*
###
related
weather_related
earthquake
###
related
###
related
aid_related
d

request
###
*empty element*
###
related
direct_report
###
related
###
related
aid_related
direct_report
other_aid
request
###
*empty element*
###
food
related
aid_related
###
*empty element*
###
related
money
aid_related
###
related
water
aid_related
direct_report
request
###
related
weather_related
transport
aid_related
direct_report
other_aid
floods
request
medical_help
###
food
related
water
aid_related
direct_report
other_aid
request
search_and_rescue
###
food
related
shelter
aid_related
direct_report
request
###
related
water
aid_related
direct_report
request
###
related
weather_related
aid_related
other_aid
other_weather
offer
###
related
water
aid_related
direct_report
request
###
related
###
related
###
related
weather_related
aid_related
direct_report
other_aid
request
other_weather
###
related
###
related
aid_related
direct_report
other_aid
request
###
food
related
water
aid_related
direct_report
request
###
related
###
related
###
food
related
death
water
aid_related
direct_

###
food
related
water
aid_related
direct_report
request
###
food
related
aid_related
direct_report
request
###
related
weather_related
security
aid_related
hospitals
cold
refugees
infrastructure_related
###
*empty element*
###
*empty element*
###
related
###
*empty element*
###
related
aid_related
other_aid
refugees
###
*empty element*
###
*empty element*
###
*empty element*
###
related
direct_report
request
###
*empty element*
###
*empty element*
###
*empty element*
###
related
###
food
related
aid_related
direct_report
request
###
related
weather_related
transport
aid_related
direct_report
other_aid
floods
request
medical_help
###
food
related
water
aid_related
buildings
direct_report
request
medical_help
###
*empty element*
###
related
###
*empty element*
###
related
aid_related
direct_report
other_aid
request
###
food
medical_products
related
clothing
shelter
death
water
aid_related
direct_report
request
medical_help
###
related
aid_related
other_aid
###
*empty element*
###
relate

medical_products
related
aid_related
direct_report
request
medical_help
###
*empty element*
###
food
medical_products
related
shelter
water
aid_related
direct_report
request
medical_help
###
related
###
*empty element*
###
*empty element*
###
food
related
shelter
aid_related
direct_report
refugees
request
###
food
related
weather_related
shelter
water
aid_related
earthquake
direct_report
request
###
food
related
water
aid_related
request
###
*empty element*
###
related
###
related
clothing
weather_related
security
transport
aid_related
direct_report
other_aid
floods
request
###
related
direct_report
request
###
*empty element*
###
related
clothing
aid_related
direct_report
request
###
related
aid_related
direct_report
other_aid
request
###
*empty element*
###
related
###
related
direct_report
request
###
related
###
related
aid_related
direct_report
other_aid
request
###
related
shelter
aid_related
direct_report
request
military
###
food
related
aid_related
direct_report
request
###
*e

###
*empty element*
###
*empty element*
###
related
weather_related
aid_related
earthquake
direct_report
other_aid
request
###
related
weather_related
aid_related
direct_report
other_aid
other_weather
###
*empty element*
###
*empty element*
###
related
aid_related
direct_report
other_aid
request
###
related
aid_related
other_aid
###
related
aid_related
direct_report
other_aid
request
###
food
medical_products
related
other_infrastructure
shelter
water
aid_related
direct_report
request
infrastructure_related
medical_help
###
*empty element*
###
related
death
aid_related
direct_report
other_aid
request
###
*empty element*
###
related
water
aid_related
direct_report
request
###
related
direct_report
###
food
related
weather_related
other_infrastructure
death
water
aid_related
storm
buildings
earthquake
direct_report
other_aid
floods
request
infrastructure_related
###
related
direct_report
###
related
weather_related
water
aid_related
direct_report
fire
request
medical_help
search_and_resc

...

request
military
###
related
weather_related
other_infrastructure
transport
floods
infrastructure_related
###
*empty element*
###
related
###
related
weather_related
floods
###
related
money
weather_related
aid_centers
aid_related
earthquake
direct_report
infrastructure_related
military
###
related
clothing
aid_related
direct_report
refugees
request
###
medical_products
related
aid_related
medical_help
###
related
###
related
aid_related
military
###
related
aid_related
refugees
military
###
*empty element*
###
related
weather_related
cold
direct_report
###
*empty element*
###
related
###
*empty element*
###
related
###
related
aid_related
medical_help
###
food
related
shelter
aid_related
direct_report
request
###
*empty element*
###
food
related
shelter
tools
aid_related
other_aid
medical_help
search_and_rescue
###
*empty element*
###
related
weather_related
aid_related
storm
other_aid
request
###
related
weather_related
shelter
aid_related
buildings
floods
###
related
aid_related
oth

other_aid
floods
medical_help
search_and_rescue
###
related
other_infrastructure
direct_report
infrastructure_related
###
related
###
*empty element*
###
medical_products
related
weather_related
aid_related
hospitals
earthquake
request
infrastructure_related
###
related
weather_related
death
aid_related
direct_report
floods
###
related
weather_related
death
transport
aid_related
storm
floods
###
*empty element*
###
related
aid_related
refugees
military
###
related
buildings
###
related
shelter
aid_related
refugees
request
###
*empty element*
###
related
aid_related
military
###
related
###
related
transport
###
*empty element*
###
*empty element*
###
related
aid_related
medical_help
###
related
death
security
aid_related
direct_report
military
###
related
###
related
###
related
###
related
###
*empty element*
###
related
###
medical_products
related
aid_related
medical_help
###
related
###
related
###
related
###
related
###
food
medical_products
related
clothing
weather_related
water

shelter
aid_related
buildings
###
food
related
weather_related
other_infrastructure
aid_related
earthquake
infrastructure_related
missing_people
###
related
###
medical_products
related
aid_related
other_aid
medical_help
###
medical_products
related
weather_related
other_infrastructure
water
aid_related
other_aid
floods
infrastructure_related
medical_help
###
*empty element*
###
*empty element*
###
related
shelter
aid_related
###
related
###
food
medical_products
related
water
aid_related
other_aid
medical_help
###
related
###
*empty element*
###
related
###
*empty element*
###
related
###
*empty element*
###
food
related
shelter
aid_related
other_aid
###
related
transport
###
related
weather_related
storm
###
related
###
related
###
related
weather_related
storm
cold
other_weather
###
related
###
food
related
clothing
aid_related
###
related
###
*empty element*
###
related
###
food
related
weather_related
aid_related
direct_report
other_weather
###
related
weather_related
shelter
aid_

medical_products
related
weather_related
shelter
aid_centers
aid_related
cold
buildings
other_aid
refugees
infrastructure_related
military
###
related
water
aid_related
direct_report
request
medical_help
###
related
death
aid_related
hospitals
infrastructure_related
medical_help
###
related
###
related
buildings
###
related
weather_related
transport
floods
###
related
weather_related
storm
earthquake
floods
other_weather
###
related
weather_related
aid_related
hospitals
cold
buildings
direct_report
fire
infrastructure_related
medical_help
search_and_rescue
###
related
###
food
medical_products
related
weather_related
shelter
death
water
tools
aid_related
other_aid
request
medical_help
other_weather
###
related
weather_related
shelter
aid_related
floods
###
related
aid_related
military
###
related
security
aid_related
other_aid
###
related
###
related
weather_related
transport
aid_related
cold
direct_report
floods
medical_help
###
related
###
related
aid_related
military
search_and_resc

elapsed time: 38.1s (0min, 38.0646sec)


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [14]:
df = df.drop('categories', axis=1)
df.head(1)

,message,original,genre,if_blank,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,False,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

drop table function [here](import logging
from sqlalchemy import MetaData
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
from sqlalchemy.ext.declarative import declarative_base)

Better way to do this, [here](https://stackoverflow.com/questions/8645250/how-to-close-sqlalchemy-connection-in-mysql)

- according to the answerer, "Engine is a **factory** for connections as well as a pool of connections, not the connection itself";
- so, to avoid the problem of couldn´t closse the connection, as other members of the poll remaining asking for transactions, the answerer recommends to use **poolclass=NulPool**;
- as we are not dealing with something that really needs a pool (only one transaction per time for us is enough), let´s do it!

In [15]:
database = create_engine('sqlite:///Messages.db', poolclass=NullPool) #, echo=True)
connection = database.connect()

#attempt to save my dataframe to SQLite
try:
    df.to_sql('Messages', database, index=False, if_exists='replace')
except ValueError:
    print('something went wrong when was writing data do SQLite')
    
connection.close()

In [16]:
spent = time() - gen_begin
print('total elapsed time: {:.1f}s ({}min, {:.4f}sec)'.format(spent, math.trunc((spent)/60), math.fmod(spent, 60)))
df.head()

total elapsed time: 41.5s (0min, 41.5418sec)


,message,original,genre,if_blank,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,False,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,False,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,False,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,False,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,False,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df.columns

Index(['message', 'original', 'genre', 'if_blank', 'related', 'request',
       'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone', 'water',
       'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees',
       'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

*this was made into a new notebook named `ETL Pipeline Test.py`, and at `etl_pipeline.py`*

---

## Later tests over the generated dataset

Count the valid registers for one column, according to a **criteria**:

- `fn_count_valids` created

In [18]:
df.shape[0]

26245

A test for the function for counting the valid labels for one column

In [19]:
total = df.shape[0]
field = 'if_blank'
count = udacourse2.fn_count_valids(dataset=df, field=field, criteria=True)
percent = 100. * (count / total)
print('{}:{} ({:.1f}%)'.format(field, count, percent))

if_blank:6317 (24.1%)


Turning this into a function

- `fn_valids_report` created

In [20]:
udacourse2.fn_valids_report(dataset=df)

###function valids_report started
  *if_blank:6317 (24.1%)


---

Labels columns, by **hierarchical structure**

*See discussion above*

In [21]:
expand_lst = ['related', 'request', 'offer', 'aid_related', 'infrastructure_related', 'weather_related', 
              'direct_report']

aid_lst = ['food', 'shelter', 'water', 'death', 'refugees', 'money', 'security', 'military', 'clothing', 
           'tools', 'missing_people', 'child_alone', 'search_and_rescue', 'medical_help', 'medical_products', 
           'aid_centers', 'other_aid']

weather_lst = ['earthquake', 'storm', 'floods', 'fire', 'cold', 'other_weather']

infrastructure_lst = ['buildings', 'transport', 'hospitals', 'electricity', 'shops', 'other_infrastructure']

To concatenate lists:

In [22]:
a = ['a', 'b']
b = ['c', 'd']
a+b

['a', 'b', 'c', 'd']

Inserting the concept into our project:
    
- `fn_count_valids` created

In [23]:
expand_list = expand_lst + aid_lst + weather_lst + infrastructure_lst

verbose = False
total = df.shape[0]
counts = []

for field in expand_list:
    count = udacourse2.fn_count_valids(dataset=df, field=field)
    percent = 100. * (count / total)
    counts.append((count, field, percent))
    if verbose:
        print('{}:{} ({:.1f}%)'.format(field, count, percent))

Order Tupples [here](https://www.pythoncentral.io/how-to-sort-a-list-tuple-or-object-with-sorted-in-python/):

- i will need it to create an **ordered report** about labels

In [24]:
sorted_tuples = sorted(counts, key=udacourse2.fn_getKey, reverse=True)

i=1
c=2
max_c=3

for cat in sorted_tuples:
    count, field, percent = cat
    print('{}-{}:{} ({:.1f}%)'.format(i, field, count, percent))
    if c > max_c:
        break
    else:
        i += 1
        c += 1

1-related:19928 (75.9%)
2-aid_related:10877 (41.4%)
3-weather_related:7304 (27.8%)


Turning this into a function:

- `fn_labels_report`created

In [45]:
udacourse2.fn_labels_report(dataset=df, 
                            max_c=11)

1-related:19928 (75.9%)
2-aid_related:10903 (41.5%)
3-weather_related:7304 (27.8%)
4-direct_report:5080 (19.4%)
5-request:4480 (17.1%)
6-other_aid:3448 (13.1%)
7-food:2930 (11.2%)
8-earthquake:2455 (9.4%)
9-storm:2448 (9.3%)
10-shelter:2319 (8.8%)
11-floods:2158 (8.2%)


---

#### Some studies about labels

*OK, I already know that **labels** are not **features**... but we may do some **critics** about the labels that exists for multi-classification*

Strong candidates labels candidates for **ignoring**:
    
>- **related** (75.0%) $\rightarrow$ too **weighty** and looks **meaningless**
>- **child_alone** (empty) $\rightarrow$ **impossible to train**, as it don´t have any valid element in the dataset
>- **request** (17.1%), **offer** (0.4%) **direct_report** (19.4%) $\rightarrow$ looks **meaningless**

---

#### Another viewpoint about these labels

If we look at them more carefully, we can find a curious pattern on them

These labels looks as they have a kind of hierarchy behind their shape, as:

First **hierarchical** class: 

>- **related**
>- **request**
>- **offer**
>- **direct_report**

And then, **related** seems to have a **Second** hierarchical class

Features for considering a training a classifier on **two layes**, or to **group** them all in main groups, as they are clearly **collinear**:

>- **aid_related** $\rightarrow$ groups aid calling (new things to add/ to do **after** the disaster)
>>- **food**
>>- **shelter**
>>- **water**
>>- **death**
>>- **refugees**
>>- **money**
>>- **security**
>>- **military**
>>- **clothing**
>>- **tools**
>>- **missing_people**
>>- **child_alone**
>>- **search_and_rescue**
>>- **medical_help**
>>- **medical_products**
>>- **aid_centers**
>>- **other_aid**
>- **weather_related** $\rightarrow$ groups what was the main **cause** of the disaster
>>- **earthquake**
>>- **storm**
>>- **floods**
>>- **fire**
>>- **cold**
>>- **other_weather**
>- **infrastructure_related** $\rightarrow$ groups **heavy infra** that was probably dammaged during the disaster
>>- **buildings**
>>- **transport**
>>- **hospitals**
>>- **electricity**
>>- **shops**
>>- **other_infrastructure**

Let´s filter & count for one **subcategory**:

In [26]:
df[df['food'] == 1].shape[0]

2930

Trying to have a union of two subcategories:

- how to make multifilters [here](https://stackoverflow.com/questions/13611065/efficient-way-to-apply-multiple-filters-to-pandas-dataframe-or-series)

- first trying two categories + **OR** clause

In [27]:
df[(df['food'] == 1) ^ (df['shelter'] == 1)].shape[0]

3659

Turning into a more automatized mode

In [28]:
a = "(df['food'] == 1) ^ (df['shelter'] == 1)"

df[eval(a)].shape[0]

3659

Preparing to turn it into a function

- `fn_cat_condenser` created

In [29]:
cat_aid_related = aid_lst
dataset = 'df'
element = '1'
opperator = '=='
condition = '^'
string = ''

for item in cat_aid_related:
    string = string + "(" + dataset + "['" + item + "'] " + opperator + " " + element + ")" + " " + condition + " "
    
string[:-3]

"(df['food'] == 1) ^ (df['shelter'] == 1) ^ (df['water'] == 1) ^ (df['death'] == 1) ^ (df['refugees'] == 1) ^ (df['money'] == 1) ^ (df['security'] == 1) ^ (df['military'] == 1) ^ (df['clothing'] == 1) ^ (df['tools'] == 1) ^ (df['missing_people'] == 1) ^ (df['child_alone'] == 1) ^ (df['search_and_rescue'] == 1) ^ (df['medical_help'] == 1) ^ (df['medical_products'] == 1) ^ (df['aid_centers'] == 1) ^ (df['other_aid'] == 1)"

---

### Tests for class `aid_related` 

Counting for the main class **aid_related**:
    
- you can see that **aid_related** have more rows registered than all subclasses counted together

- this is not about training a **Machine Learning**, this is about **database data consistency**

*Just think in this way: if something is labelled as **aid_related**, so every data under it may be contained by **aid_related**. So next step, we need to correct this thing, turning **aid_related = 1** for all of them*

In [30]:
df_aid_main = df[df['aid_related'] == 1]
df_aid_main.shape[0]

10877

Filtering for **main**, without any **sub-category** registered:

- this is **empty data** as the main category is checked, but the subcategory is not!

In [31]:
#fn_cat_condenser(subset='aid', opperation='main_not_sub')

In [32]:
main_not_sub = df[eval(udacourse2.fn_cat_condenser(subset='aid', 
                                                   name='df',
                                                   opperation='main_not_sub')[0])]
main_not_sub.shape[0]

3515

Testing the function for grouping all **subclasses**

- counting for all subclasses of **aid_related*

- filtering for all **subcategories** with any register:

In [33]:
#fn_cat_condenser(subset='aid', opperation='all_sub')

In [34]:
all_aid_subsets = df[eval(udacourse2.fn_cat_condenser(subset='aid',
                                                      name='df',
                                                      opperation='all_sub')[0])]
all_aid_subsets.shape[0]

7388

Filtering for **all empty** subcategories:

In [35]:
#fn_cat_condenser(subset='aid', opperation='sub_not_main')

In [36]:
sub_not_main = df[eval(udacourse2.fn_cat_condenser(subset='aid',
                                                   name='df',
                                                   opperation='empty_sub')[0])]
sub_not_main.shape[0]

0

---

#### Inconsistency detected

1. Just consider that our **labels** have a kind of **hierarchical structure**

2. Before running a Machine Learning **Classifier**, we can procede some **mechanical opperations** for correcting database **data inconsistencies**

3. The theoretical support for this opperations resides on **database theory** (e.g. if something is a subclass of other thing, so, the class must be setted on for each of the rows that are setted for at least one item of the subclass) 



Filtering for **empty main** with any **subcategory** registered:

>- this is a **database inconsistency** 
>- as if a **subgroup** is valid, so the **main** group must be valid too!
>- about database **normal forms**, please read [here](https://www.guru99.com/database-normalization.html)

In [37]:
#fn_cat_condenser(subset='aid', opperation='sub_not_main')

In [38]:
sub_not_main = df[eval(udacourse2.fn_cat_condenser(subset='aid',
                                                   name='df',
                                                   opperation='sub_not_main')[0])]
sub_not_main.shape[0]

26

Correcting our **data inconsistency**:

>- considering `opp` is only a **filter** for columns that have any value in sublabels
>- `df[eval(opp[0])]` gives our dataset rows for this filter
>- `df[eval(opp[0])].index` is only about the address to find these rows
>- finally, we need to correct `opp[1]]`, that is the name of the column, by the value `1`

In [39]:
opp = udacourse2.fn_cat_condenser(subset='aid',
                                  name='df',
                                  opperation='sub_not_main')

#df[eval(opp[0])]
#df.loc[df[eval(opp[0])].index, opp[1]] = 1 #I don´t want to correct it now!

sub_not_main = df[eval(udacourse2.fn_cat_condenser(subset='aid', opperation='sub_not_main')[0])]
sub_not_main.shape[0]

26

Turning this into a function

- function `fn_croup_check` created

In [40]:
df_aid = udacourse2.fn_group_check(dataset=df,
                                   subset='aid',
                                   correct=True, 
                                   shrink=True, 
                                   shorten=True, 
                                   verbose=True)
df_aid.head(5)

###function group_check started
  - count for main class:aid_related, 10877 entries
  - for main, without any sub-categories,  3515 entries
  - for subcategories,  7388 entries
  - for lost parent sub-categories,  26 entries
    *correcting, new count: 0 entries
    *shrinking, dataset now have: 21 columns
    *shortening, dataset now have: 10903 lines
elapsed time: 0.1099s


,message,genre,if_blank,aid_related,food,shelter,water,death,refugees,money,...,military,clothing,tools,missing_people,child_alone,search_and_rescue,medical_help,medical_products,aid_centers,other_aid
id,,,,,,,,,,,,,,,,,,,,,
7,Is the Hurricane over or is it not over,direct,False,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,False,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
16,"Please, we need tents and water. We are in Sil...",direct,False,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,I am in Croix-des-Bouquets. We have health iss...,direct,False,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
20,"There's nothing to eat and water, we starving ...",direct,False,1,1,0,1,0,0,0,...,0,0,0,0,0,0,1,1,0,1


---

### About rows:
    
- We have 40 columns, and some rows that cannot be used to **train** any model (all their features are blank)

All the dataframe:

In [41]:
df.shape[0]

26245

Blank labels:
    
- it's not a good idea training with this data;

- they have **zero** classification (no labels at all!)

In [42]:
df[df['if_blank']].shape[0]

6317

Labels with some content:

In [43]:
df[~df['if_blank']].shape[0]

19928

---

## Test area (can be removed later)

In [44]:
raise Exception('Test area')

Exception: Test area

In [ ]:
full_path = 'c://host/pyprog/project/udacourse/categories.csv'
full_path = 'c://host/pyprog/project/udacourse/messages.csv'
last_one = full_path.rfind('/')
full_path[last_one+1:-4]

In [ ]:
categories.index.name = 'categories'
categories.index.name

In [ ]:
uma = 0.70
duas = 0.305
primeira = 0.35 #0.51
segunda = 0.67 #0.74

delta = ((uma - duas) * primeira) + (duas * segunda)
delta

In [ ]:
uma = 0.70
duas = 0.305
primeira = 0.70
segunda = 0.88

original = ((uma - duas) * primeira) + (duas * segunda)
original

In [ ]:
primeira = 0.51
segunda = 0.74

alfa = ((uma - duas) * primeira) + (duas * segunda)
alfa

In [ ]:
def fn_teste(dataset):
    #print(dataset.iloc[0])
    print(locals())

In [ ]:
fn_teste(dataset=df)